In [23]:
import boto3
import xml.etree.ElementTree as ET
from botocore.exceptions import ClientError
from tqdm.notebook import tqdm

def list_xml_files(bucket_name, prefix=''):
    s3 = boto3.client('s3')
    xml_files = []

    try:
        paginator = s3.get_paginator('list_objects_v2')
        for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
            if 'Contents' in page:
                for obj in page['Contents']:
                    if obj['Key'].lower().endswith('.xml'):
                        xml_files.append(obj['Key'])
    except ClientError as e:
        print(f"Error listing files: {e}")

    return xml_files

def read_xml_file(bucket_name, file_key):
    s3 = boto3.client('s3')
    try:
        response = s3.get_object(Bucket=bucket_name, Key=file_key)
        xml_content = response['Body'].read().decode('utf-8')
        return xml_content
    except ClientError as e:
        print(f"Error reading file {file_key}: {e}")
        return None

def extract_info(xml_content):
    root = ET.fromstring(xml_content)
    
    impression = root.find('.//Impression')
    impression_value = impression.text.strip() if impression is not None and impression.text is not None else "Not found"
    
    tracking_events = root.findall('.//Tracking')
    tracking_values = {event.attrib['event']: event.text.strip() for event in tracking_events if event.text is not None}
    
    click_through = root.find('.//ClickThrough')
    click_through_value = click_through.text.strip() if click_through is not None and click_through.text is not None else "Not found"
    
    click_tracking = root.find('.//ClickTracking')
    click_tracking_value = click_tracking.text.strip() if click_tracking is not None and click_tracking.text is not None else "Not found"
    
    return {
        'Impression': impression_value,
        'TrackingEvents': tracking_values,
        'ClickThrough': click_through_value,
        'ClickTracking': click_tracking_value
    }

data = {}
def main():
    bucket_name = 'amac-adressable-tv'
    prefix = 'vast2.0/'  # Optional: specify a prefix if XML files are in a specific directory

    xml_files = list_xml_files(bucket_name, prefix)
    print("xml_files_len:", len(xml_files))
    # data = {}
    for file_key in tqdm(xml_files):
        # print(f"Processing file: {file_key}")
        xml_content = read_xml_file(bucket_name, file_key)
        if xml_content:
            info = extract_info(xml_content)
            data[file_key.split("/")[-1][:-4]]=info
            # print("Extracted Information:")
            # print(f"Impression: {info['Impression']}")
            # print("Tracking Events:")
            # for event, value in info['TrackingEvents'].items():
            #     print(f"  {event}: {value}")
            # print(f"ClickThrough: {info['ClickThrough']}")
            # print(f"ClickTracking: {info['ClickTracking']}")
            # print("\n")

if __name__ == "__main__":
    main()

xml_files_len: 0


0it [00:00, ?it/s]

In [18]:
data2=[]
for k, v in data.items():
    d= {
        'pubid':k,
        'impression':v['Impression'],
        'ClickThrough':v['ClickThrough'],
        'ClickTracking':v['ClickTracking'],
    }
    # for ev in ["ClickThrough", "ClickTracking", "midpoint", "complete", "skip", "verificationNotExecuted"]
    for k1, v1 in v['TrackingEvents'].items():
        d[k1]=v1
    data2.append(d)

In [19]:
import pandas as pd 

df = pd.DataFrame(data2)

In [20]:
df.replace(['', 'Mobile', 'Not found'], pd.NA, inplace=True)
df

,pubid,impression,ClickThrough,ClickTracking,start,midpoint,complete,skip,verificationNotExecuted
0,761086d,https://ad.doubleclick.net/ddm/trackimp/N5295....,https://ad.doubleclick.net/ddm/trackclk/N5295....,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,761086m,https://ad.doubleclick.net/ddm/trackimp/N5295....,https://ad.doubleclick.net/ddm/trackclk/N5295....,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Campagne_BPCE_Caisse_Eparge_JO_Mai_Juin_742644,https://ad.doubleclick.net/ddm/trackimp/N30980...,https://ad.doubleclick.net/ddm/trackclk/N30980...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,FR_ABEILLE_0001_015_F,https://viewbox.realytics.net/?KV=performance&...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN
4,FR_ALPA_ABAS_EPAR_0001_015_F,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...
96,pubid_FR_HAVM_BAFT_CA MP_0001_020_F,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN
97,pubid_FR_ORSG_SHIE_SHIN_0005_020_F,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
98,pubid_FR_ORSG_SHIE_SHIN_0006_020_F,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
99,pubid_FR_TEST_ COF_0001_020_F,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [11]:
df.impression.value_counts()

impression
https://ad.doubleclick.net/ddm/trackimp/N1053267.4969466ALTICERMCSPORT/B32038916.395596185;dc_trk_aid=586955458;dc_trk_cid=215617721;ord=[timestamp];dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=;gdpr=${GDPR};gdpr_consent=${GDPR_CONSENT_755};ltd=;dc_tdv=1         7
https://ad.doubleclick.net/ddm/trackimp/N1053267.4969466ALTICERMCSPORT/B32038916.395596185;dc_trk_aid=586955458;dc_trk_cid=215617721;ord=[timestamp];dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=;gdpr=${GDPR};gdpr_consent=${GDPR_CONSENT_755};ltd=;dc_tdv=1?        6
https://ad.doubleclick.net/ddm/trackimp/N1955507.143981BFMTV.FR/B32217359.398948487;dc_trk_aid=590475907;dc_trk_cid=218766069;ord=[timestamp];dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=;gdpr=${GDPR};gdpr_consent=${GDPR_CONSENT_755};ltd=;dc_tdv=1?               3
https://ad.doubleclick.net/ddm/trackimp/N450807.2268505ALTICEMEDIAFR/B32242749.397502402;dc_trk_aid=589290924;dc_trk_cid=217220254;ord=[timestamp];dc_lat=;dc_rdi

In [22]:
df.ClickThrough.value_counts()

ClickThrough
https://ad.doubleclick.net/ddm/trackclk/N1053267.4969466ALTICERMCSPORT/B32038916.395596185;dc_trk_aid=586955458;dc_trk_cid=215617721;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=;ltd=;dc_tdv=1                                                                 13
https://ad.doubleclick.net/ddm/trackclk/N1955507.143981BFMTV.FR/B32217359.398948487;dc_trk_aid=590475907;dc_trk_cid=218766069;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=;ltd=;dc_tdv=1                                                                         3
https://ad.doubleclick.net/ddm/trackclk/N450807.2268505ALTICEMEDIAFR/B32242749.397502402;dc_trk_aid=589290924;dc_trk_cid=217220254;dc_lat=;dc_rdid=;tag_for_child_directed_treatment=;tfua=;ltd=;dc_tdv=1                                                                    3
https://ad.doubleclick.net/ddm/trackclk/N5295.4108859ALTICEMEDIA/B32234518.400059322;dc_trk_aid=592256120;dc_trk_cid=219241151;dc_lat=;dc_rdid=;tag_for_child_directed_treatme

In [32]:
info = {}
loudness_flag = False
for line in text.split("\n"):
    if "Duration" in line:
        match = re.search(r"Duration: (\d{2}):(\d{2}):(\d{2})\.(\d{2})", line)
        if match:
            hours, minutes, seconds, milliseconds = map(int, match.groups())
            info["duration_seconds"] = hours * 3600 + minutes * 60 + seconds + milliseconds / 100
    elif "bitrate" in line:
        match = re.search(r"bitrate: (\d+) kb/s", line)
        if match:
            info["bitrate_kbps"] = int(match.group(1))
    elif "Stream" in line:
        width_height_match = re.search(r"(\d+)x(\d+)", line)
        if width_height_match:
            info["width"], info["height"] = map(int, width_height_match.groups())
    elif "Summary:" in line:
        loudness_flag = True
    elif loudness_flag and "I:" in line:
        info["integrated_loudness"] = float(line.split('I:')[1].strip().split(' ')[0])
    elif loudness_flag and 'LRA:' in line:
        info["loudness_range"] = float(line.split('LRA:')[1].strip().split(' ')[0])
        

info

{'duration_seconds': 30.0, 'width': 0, 'height': 1}

In [ ]:
from glob import glob
from tqdm import tqdm
l={}
for file in tqdm(glob(r"C:\Users\mohammed\Desktop\workdir\app\data\video_simples\*.mp4")):
    # if len(l)==2:
    #     break
    d = get_video_info(file)
    if not d: continue
    if d["integrated_loudness"]<=-23:
       l[file] = d

In [7]:
l

{'C:\\Users\\mohammed\\Desktop\\workdir\\app\\data\\video_simples\\FR_GROU_MONS_SOLA_0004_015_F_MP4_1709298602.mp4': {'duration_seconds': 15.0,
  'bitrate_kbps': 10867,
  'width': 1920,
  'height': 1080,
  'integrated_loudness': -23.0,
  'loudness_range': 1.7},
 'C:\\Users\\mohammed\\Desktop\\workdir\\app\\data\\video_simples\\FR_GROU_MONS_SOLA_0005_015_F_MP4_AVEC_PROMO_1709298526.mp4': {'duration_seconds': 15.0,
  'bitrate_kbps': 15628,
  'width': 1920,
  'height': 1080,
  'integrated_loudness': -23.0,
  'loudness_range': 1.6},
 'C:\\Users\\mohammed\\Desktop\\workdir\\app\\data\\video_simples\\pubid_FR_AUST_CICC_PETZ_0001_027_F_1709652921_kantar_1920x1080.mp4': {'duration_seconds': 27.0,
  'bitrate_kbps': 2108,
  'width': 1920,
  'height': 1080,
  'integrated_loudness': -23.1,
  'loudness_range': 5.0},
 'C:\\Users\\mohammed\\Desktop\\workdir\\app\\data\\video_simples\\pubid_FR_BETC_GLEC_LECL_0362_030_F_1710150277.mp4': {'duration_seconds': 30.0,
  'bitrate_kbps': 10513,
  'width': 192

In [13]:
import subprocess

def normalize_audio_loudness(input_file, output_file, target_loudness=-22):
    """
    Normalizes the audio loudness of a video file using FFmpeg.

    Args:
        input_file (str): The path to the input video file.
        output_file (str): The path to the output video file.
        target_loudness (float): The desired integrated loudness level in LUFS.

    Raises:
        Exception: If there's an error running FFmpeg.
    """

    command = [
        FFMPEG_BINARY,
        "-i", input_file,
        "-af", f"loudnorm=I={target_loudness}:LRA=7:TP=-3", # f"loudnorm=I={target_loudness}:TP=-1.5:LRA=11"
        "-c:v", "copy",
        output_file, "-y"
    ]

    result = sp.run(command, capture_output=True, text=True)

    if result.returncode != 0:
        raise Exception(f"Error running FFmpeg: {result.stderr}")

    print("Audio normalized successfully.")

# Example usage:
input_file = "C:\\Users\\mohammed\\Desktop\\workdir\\app\\data\\video_simples\\FR_BUZZ_GIDA_PPUP_0005_015_F_IMD_1710342988.mp4"
output_file = "C:\\Users\\mohammed\\Desktop\\workdir\\app\\data\\video_simples\\normalized_video.mp4"
normalize_audio_loudness(input_file, output_file)

Exception: Error running FFmpeg: ffmpeg version 2024-02-26-git-a3ca4beeaa-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --pkg-config=pkgconf --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-li

In [7]:
get_video_info(output_file)

Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]


{'duration_seconds': 15.1,
 'bitrate_kbps': 1930,
 'width': 1920,
 'height': 1080,
 'integrated_loudness': -22.5,
 'loudness_range': 1.8}

In [1]:
import sys, os, io, json
import subprocess as sp
cmd_timeout=10*60*60
import re
FFMPEG_BINARY="C:/ffmpeg/bin/ffmpeg"

def get_video_info(file_path: str):
    """Get video information using ffmpeg."""
    print("Get video information using ffmpeg [START]")
    try:
        command = [
            FFMPEG_BINARY,
            "-i", file_path,
            "-af", "ebur128=peak=true",
            "-f", "null",
            "-"
        ]
        result = sp.run(command, capture_output=True, text=True, check=True)
        
        info = {}
        loudness_flag = False
        with open(file_path+".log", "w") as f:
            f.write(result.stderr)
        for line in result.stderr.splitlines():
            if "Duration" in line:
                match = re.search(r"Duration: (\d{2}):(\d{2}):(\d{2})\.(\d{2})", line)
                if match:
                    hours, minutes, seconds, milliseconds = map(int, match.groups())
                    info["duration_seconds"] = hours * 3600 + minutes * 60 + seconds + milliseconds / 100

                fps_match = re.search(r'(\d+)\s+fps', line)
                if fps_match:
                    info["fps"] = int(fps_match.group(1))

            if "bitrate" in line:
                match = re.search(r"bitrate: (\d+) kb/s", line)
                if match:
                    info["bitrate_kbps"] = int(match.group(1))
            elif "Stream" in line and "Video:" in line:
                width_height_match = re.search(r", (\d+)x(\d+)", line)
                if width_height_match:
                    info["width"], info["height"] = map(int, width_height_match.groups())
            elif "Summary:" in line:
                loudness_flag = True
            elif loudness_flag and "I:" in line:
                info["integrated_loudness"] = float(line.split('I:')[1].strip().split(' ')[0])
            elif loudness_flag and 'LRA:' in line:
                info["loudness_range"] = float(line.split('LRA:')[1].strip().split(' ')[0])
        
        print("Get video information using ffmpeg [DONE]")
        return info
    
    except sp.CalledProcessError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    return None

def bytes_to_str(st):
    if st is None:
        return None
    if isinstance(st, bytes):
        try:
            return  st.decode('utf-8').strip()
        except UnicodeDecodeError:
            return  st.decode('latin-1').strip()
    return str(st).strip()

def exec_cmd(cmd, timeout=cmd_timeout):
    """Exécution d'une commande shell dans un process séparé.

    Arguments:
        cmd {str} -- Commande shell à exécuter.

    Returns:
        entier -- Code retour de l'exécution de la commande :
                    str type - Succès: le stdout du commande
                    None - Echec: avec afiche sur le log l'erreur
    """
    print(f"Appel de la commande: {cmd}")
    stdout, stderr, process = None, None, None
    process = sp.Popen(str(cmd), shell=True, stderr=sp.PIPE, stdout=sp.PIPE)
    try:
        stdout, stderr = process.communicate(timeout=timeout)
    except sp.TimeoutExpired:
        # Dans cet exemple, nous lançons une commande externe avec `subprocess.Popen` et utilisons la méthode `communicate()` avec un timeout en secondes. 
        # Si le processus ne se termine pas avant l'expiration du délai, nous tuons le processus avec `process.kill()` et récupérons les sorties standard et d'erreur.
        process.kill()
        stdout, stderr = process.communicate()
        stderr=bytes_to_str(stderr)
        stdout=bytes_to_str(stdout)
        print(f"Le processus a expiré après {timeout} secondes" +
              "\n==> stderr: \n"+stderr+"\n==> stdout: \n"+stdout)
        return False, None, None
    except sp.CalledProcessError as e:
        print(f"Erreur lors de l'appel à la commande: {e}")
        return False, None, None
    except Exception as e:
        print(f"Unexpected lors de l'appel à la commande: {e}")
        return False, None, None

    returncode = int(process.returncode)
    stderr=bytes_to_str(stderr)
    stdout=bytes_to_str(stdout)
    if returncode != 0 and stderr is not None and stderr != '':
        print("Erreur lors de l'appel à la commande; returncode="+str(returncode) +
              "\n==> stderr: \n"+stderr+"\n==> stdout: \n"+stdout)
        return False, stdout, stderr
    # print("\n==> stderr: \n"+bytes_to_str(stderr)+"\n==> stdout: \n"+bytes_to_str(stdout))
    print(f"Succès de la commande {cmd}")
    return True, stdout, stderr

In [2]:

# input_video_path=r"C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4"
input_video_path=r"C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4"
expected_duration=20

output_video_path = os.path.join(os.path.dirname(os.path.abspath(input_video_path)),"file_out.mp4")
silence_video_path = os.path.join(os.path.dirname(os.path.abspath(input_video_path)),"file_silence.mp4")

print(get_video_info(input_video_path))

import cv2
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
cap.release()
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count

Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]
{'duration_seconds': 19.98, 'bitrate_kbps': 16699, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.3, 'loudness_range': 2.5}
frame_rate: 23.976023976023978; frame_count: 479, expected_frame_count: 479


True

In [39]:
FFMPEG_BINARY="ffmpeg"
cmd=f'{FFMPEG_BINARY} -i "{video_path}" '
for w, h in [(640,360), (854,480), (1280,720), (1920,1080)]:
    cmd+=f'''-f mp4 -s {w}x{h} -vf scale={w}:{h} -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a {'128k' if h<=480 else '192k'} -ss 00:00:00.000 -t 00:00:{expected_duration}.000 "C:/Users/mohammed/Desktop/workdir/app/assets/{h}p.mp4" -y '''
    # cmd+=f''' -f mp4 -c:v mpeg2video -pix_fmt yuv422p -profile:v 0 -s {w}x{h} -vf "scale={w}:{h}" -aspect 16:9 -r 25 -g 12 -c:a aac -b:a {'128k' if h<=480 else '192k'} "C:/Users/mohammed/Desktop/workdir/app/assets/{h}p.mp4" -y '''

flag, v1, v2 = exec_cmd(str(cmd).replace("\n", " "))
flag

Appel de la commande: ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4" -f mp4 -s 640x360 -vf scale=640:360 -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a 128k -ss 00:00:00.000 -t 00:00:20.000 "C:/Users/mohammed/Desktop/workdir/app/assets/360p.mp4" -y -f mp4 -s 854x480 -vf scale=854:480 -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a 128k -ss 00:00:00.000 -t 00:00:20.000 "C:/Users/mohammed/Desktop/workdir/app/assets/480p.mp4" -y -f mp4 -s 1280x720 -vf scale=1280:720 -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a 192k -ss 00:00:00.000 -t 00:00:20.000 "C:/Users/mohammed/Desktop/workdir/app/assets/720p.mp4" -y -f mp4 -s 1920x1080 -vf scale=1920:1080 -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a 192k -ss 00:00:00.000 -t 00:00:20.000 "C:/Users/mohammed/Desktop/workdir/app/assets/1080p.mp4" -y 
Succès de la commande ffmpeg -i "C:\Users\mohammed

True

In [6]:
FFMPEG_BINARY=r"C:\ffmpeg\bin/ffmpeg"
cmd=f'{FFMPEG_BINARY} -i "{video_path}" '
# cmd+=f'-filter:v "setpts=1.001*PTS,fps=25" ' # 20/19.98 = 1.001*PTS
# cmd+=f'-filter:v "fps=25,setpts=(19.98/20)*PTS" '
# cmd+=f'-filter_complex "[0:v]fps=25,split[v1][v2];[v1]trim=duration=19.98[v1trim];[v2]trim=start=19.98,setpts=PTS-STARTPTS[v2trim];[v1trim][v2trim]concat=n=2:v=1:a=0" '
# cmd+=f''' -filter_complex "[0:v]fps=25,select='if(gte(n,499),499,n)',loop=500:1:499[v]" -map "[v]" '''
# cmd+='-vf "fps=25,pad=width=iw:height=ih:color=black:duration=20" '
# cmd+=f''' -vf "fps=25,select='if(gte(n,499),499,n)',setpts=N/25/TB" -af "apad" '''
cmd+=f''' -vf "fps=25" -af "apad" -r 25 -g 12 '''
cmd+=f'-ss 00:00:00.000 -t 00:00:{expected_duration}.000 '
cmd+=f'-c:v libx264 -preset slow -crf 23 -c:a aac -b:a 128k '
cmd+=r'"C:\Users\mohammed\Desktop\workdir\app\assets\1080p.mp4" -y'
# print(cmd)
flag, v1, v2 = exec_cmd(str(cmd).replace("\n", " "))
flag

Appel de la commande: C:\ffmpeg\bin/ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4"  -vf "fps=25" -af "apad" -r 25 -g 12 -ss 00:00:00.000 -t 00:00:20.000 -c:v libx264 -preset slow -crf 23 -c:a aac -b:a 128k "C:\Users\mohammed\Desktop\workdir\app\assets\1080p.mp4" -y
Succès de la commande C:\ffmpeg\bin/ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4"  -vf "fps=25" -af "apad" -r 25 -g 12 -ss 00:00:00.000 -t 00:00:20.000 -c:v libx264 -preset slow -crf 23 -c:a aac -b:a 128k "C:\Users\mohammed\Desktop\workdir\app\assets\1080p.mp4" -y


True

In [9]:
input_video_path=r"C:\Users\mohammed\Desktop\workdir\app\assets\1080p.mp4"
# input_video_path=r"C:\Users\mohammed\Desktop\workdir\FR_FOCN_GIFP_FORM_0001_030_F_30.07.mp4"
output_video_path=r"C:\Users\mohammed\Desktop\workdir\app\assets\1080p_2.mp4"
silence_video_path = os.path.join(os.path.dirname(os.path.abspath(output_video_path)),"file_silence.mp4")
if os.path.exists(output_video_path):
    os.remove(output_video_path)
if os.path.exists(silence_video_path):
    os.remove(silence_video_path)
expected_duration=20
print(get_video_info(input_video_path))

import cv2
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
# cap.release()
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count

Get video information using ffmpeg [START]
Unexpected error: [WinError 2] The system cannot find the file specified
None
frame_rate: 25.0; frame_count: 499, expected_frame_count: 500


False

In [10]:
expected_frame_count = expected_duration * frame_rate
incorect_frame_count = expected_frame_count-frame_count
print("incorect_frame_count:",incorect_frame_count)

# Create a VideoWriter object to write the new video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(silence_video_path, fourcc, frame_rate, (width, height))

# Write the original frames to the new video except the last one
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

if expected_frame_count < frame_count:
    for _ in range(int(expected_frame_count)):
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
else:
    print("1")
    cnt=0
    # Write the original frames to the new video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        cnt+=1

    # Read the last frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count - 1)
    ret, last_frame = cap.read()

    # Write the duplicated last frame to the new video
    for _ in range(int(incorect_frame_count)):
        out.write(last_frame)
        cnt+=1

print("cnt:",cnt)
# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()


incorect_frame_count: 1.0
1
cnt: 500


In [24]:
FFMPEG_BINARY="ffmpeg"
cmd = f"""{FFMPEG_BINARY} -i "{silence_video_path}" -i "{input_video_path}" -c copy -map 0:v:0 -map 1:a:0 -y "{output_video_path}" """
# cmd = f"""{FFMPEG_BINARY} -i "{silence_video_path}" -i "{input_video_path}" -c copy -map 0:v -map 1:a -y "{output_video_path}" """

flag, _, _ = exec_cmd(str(cmd).replace("\n", " "))
flag

Appel de la commande: ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\file_silence.mp4" -i "C:\Users\mohammed\Desktop\workdir\app\assets\1080p.mp4" -c copy -map 0:v:0 -map 1:a:0 -y "C:\Users\mohammed\Desktop\workdir\app\assets\1080p_2.mp4" 
Succès de la commande ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\file_silence.mp4" -i "C:\Users\mohammed\Desktop\workdir\app\assets\1080p.mp4" -c copy -map 0:v:0 -map 1:a:0 -y "C:\Users\mohammed\Desktop\workdir\app\assets\1080p_2.mp4" 


True

In [25]:
expected_duration=20
print(get_video_info(output_video_path))

import cv2
cap = cv2.VideoCapture(output_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
cap.release()
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count

Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]
{'duration_seconds': 20.0, 'bitrate_kbps': 10216, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.3, 'loudness_range': 2.4}
frame_rate: 25.0; frame_count: 500, expected_frame_count: 500


True